<font face='Calibri' size='2'> <i>eSBAE - Notebook Series - Part 4, version 0.1, April 2023. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![titre](images/header.png)

# IV - Augmentation des données eSBAE
### Exécutez divers algorithmes de détection de changement sur des données de séries chronologiques précédemment extraites
-------

Ce bloc-notes vous guide tout au long du processus d'exécution de divers algorithmes de détection de modification pour un ensemble de points à l'aide de [Google's Earth Engine](https://earthengine.google.com/) ainsi que des routines Python. Le script est optimisé pour traiter des milliers de points et utilisera la parallélisation pour extraire efficacement les informations de la plateforme.

**Vous aurez besoin de**:
- un compte Earth Engine valide ([inscrivez-vous ici](https://code.earthengine.google.com/register))
- avoir exécuté avec succès le Notebook 3 de la série de notebooks eSBAE

**Ce notebook fonctionne mieux sur une instance m16**

### 1 - Importer des bibliothèques (*exécuter uniquement cette cellule*)

Cette cellule nous fournira les fonctionnalités dont nous avons besoin pour exécuter les cellules suivantes du notebook.

In [1]:
# initialize EE    
import ee
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    
from sampling_handler import DataAugmentation

/usr/local/share/jupyter/kernels/venv-eSBAE_notebooks/venv/lib/python3.8/site-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Définitions de la période

In [2]:
# Directory where output and temp files will go
esbae = DataAugmentation(
    
    # your project name, as set in previous notebooks
    project_name = 'esbae_congo',

    # start of calibration period (mainly for bfast)
    calibration_start = '2015-01-01',  # YYYY-MM-DD format

    # Actual period of interest, i.e. monitoring period
    monitor_start =  '2020-01-01',  # YYYY-MM-DD format
    monitor_end   =  '2023-01-01',  # YYYY-MM-DD format

    # select the band for univariate ts-analysis (has to be inside bands list)
    ts_band = 'ndfi'
)

INFO: Using existing project directory at /home/sepal-user/module_results/esbae/esbae_congo
INFO: Using existent config file from project directory /home/sepal-user/module_results/esbae/esbae_congo


### 3- Paramètres de base

In [3]:
# select basic pre-processing options
esbae.outlier_removal =     True
esbae.smooth_ts =           True

# Select the algorithms to run
esbae.run_cusum =           True
esbae.run_bfast =           True
esbae.run_ts_metrics =      True
esbae.run_bs_slope =        True
esbae.run_jrc_nrt =         True
esbae.run_ccdc =            True
esbae.run_land_trendr =     False     # not yet implemented
esbae.run_global_products = True

### 4- Paramètres avancés

In [4]:
esbae.bfast = {
    'run': esbae.run_bfast,
    'start_monitor': esbae.monitor_start,
    'freq': 365,
    'k': 3,
    'hfrac': 0.25,
    'trend': True,
    'level': 0.05,
    'backend': 'python'
}

esbae.cusum = {
    'run': esbae.run_cusum,
    'nr_of_bootstraps': 1000
}

esbae.bs_slope = {
    'run': esbae.run_bs_slope,
    'nr_of_bootstraps': 1000
}

esbae.ts_metrics = {
    'run': esbae.run_ts_metrics,
    'bands': ['red', 'nir', 'swir1', 'swir2', 'ndfi', 'brightness', 'greenness', 'wetness'],
    'metrics': ['mean', 'stddev', 'min', 'max'],  # DO NOT CHANGE YET
    'outlier_removal': False,
    'z_threshhold': 3
}

esbae.ccdc = {
    'run': esbae.run_ccdc,
    'breakpointBands': ['green', 'red', 'nir', 'swir1', 'swir2'],
    'tmaskBands': ['green', 'swir2'],
    'minObservations': 6,
    'chiSquareProbability': 0.99,
    'minNumOfYearsScaler': 1,
    'dateFormat': 2,
    'lambda': 20,
    'maxIterations': 1000
}

esbae.land_trendr = {
    'run': esbae.run_land_trendr,
    'maxSegments': 6,
    'spikeThreshold': 0.9,
    'vertexCountOvershoot': 3,
    'preventOneYearRecovery': True,
    'recoveryThreshold': 0.25,
    'pvalThreshold': 0.05,
    'bestModelProportion': 0.75,
    'minObservationsNeeded': 3
}

esbae.jrc_nrt = {
    'run': esbae.run_jrc_nrt
}

esbae.global_products = {
    'run': esbae.run_global_products,
    'gfc': True,
    'tmf': True,
    'tmf_years': True,
    'esa_lc20': True,
    'copernicus_lc': True,
    'esri_lc': True,
    'lang_tree_height': True,
    'potapov_tree_height': True,
    'elevation': True,
    'dynamic_world_tree_prob': True,
    'dynamic_world_class_mode': True
}

esbae.py_workers = 100
esbae.ee_workers = 20

### 5 - Exécutez l'augmentation des données

In [5]:
esbae.augment()

INFO: Verifying parameter settings...
INFO: Initializing data augmentation routine...
INFO: Accumulating batch files of 1/1...
INFO: Running the data augmentation routines on 3570 points...
INFO: Cleaning the time-series from outliers.
INFO: Outlier removal finished in: 0:00:07.839647
INFO: Smoothing the time-series with a rolling mean.
INFO: Time-series smoothing finished in: 0:00:12.885177
INFO: Creating a subset of the time-series according to the analysis period.
INFO: Time-series subsetting finished in: 0:00:02.442055
INFO: Running CCDC
INFO: Running the B-FAST algorithm on current batch of points.
INFO: BFAST finished in: 0:00:04.827528
INFO: Running the CuSum algorithm on current batch of points.
INFO: CuSum finished in: 0:00:37.311171
INFO: Running the time-scan on current batch of points.
INFO: Time-scan metrics for band red finished in: 0:00:01.814901
INFO: Time-scan metrics for band nir finished in: 0:00:02.655914
INFO: Time-scan metrics for band swir1 finished in: 0:00:03.2